<b>Kagawa et al. Predicting elevated natriuretic peptide in chest radiography: Emerging utilization gap for artificial intelligence. European Heart Journal - Imaging Methods and Practice, qyae064, https://doi.org/10.1093/ehjimp/qyae064<br>
<br>
Python sample code: the final model (ensemble model), BNP cut-off 200 pg/mL<br></b>
<br>
Version 1.0.0 July 05, 2024: First release.<br>
<br>
The sample codes are utilized in Google Colab or Jupyter Notebook.<br>
<br>
In Google Colab or Jupyter Notebook, either click the arrow on the cell or select the cell and use the appropriate shortcut to execute it (Shift + Enter).<br>

1. If you utilize in Google Colab, mount Google Colab.<br>
2. Install tensorflow_addons.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

!pip install tensorflow_addons
!pip install vit_keras

2. Import libraries.<br>
Please ignore TFA warning.<br>

In [1]:
import tensorflow as tf
import tensorflow_addons as tfa
import numpy as np
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from PIL import Image
import matplotlib.pyplot as plt
import os
from vit_keras import vit                                    
from vit_keras.vit import preprocess_inputs

3. Move or copy the chest radiography image you want to test by specifying the 'image_file_path'.<br>
Example:<br>
image_file_path = '/content/drive/MyDrive/image_file.png'<br>

In [2]:
# feature_branch. DELETE develop or main branch

'''
image_file_path = '/content/drive/MyDrive/image_file.png'
'''

#image_file_path = 'C:/Users/egkag/Dropbox/GitHub_public_ekagawa007/Zenodo/MichikoSakamoto91F20220517Cre373Hb115AHF1BNP1518_2.png'
#image_file_path = 'C:/Users/egkag/Dropbox/GitHub_public_ekagawa007/Zenodo/AtsukoSuzawa85F20220519Cre177Hb102BNP2296_resizedSq.png'
#image_file_path = 'C:/Users/egkag/Dropbox/GitHub_public_ekagawa007/Zenodo/MichihisaUeda63M20220531Cre069Hb137BNP57.png'
image_file_path = 'C:/Users/egkag/Dropbox/GitHub_public_ekagawa007/Zenodo/AkiraKubo85M20181224Cre087Hb138BNP236.png'


In [3]:
image_file_path = '/content/drive/MyDrive/image_file.png'

4. Visualize the image.<br>

In [ ]:
BACKGROUND_COLOR = (0, 0, 0) #black

def load_and_preprocess_image(path, target_size):   
    try:
        im = Image.open(path).convert("RGB")
        width, height = im.size
        if width > height:
            result = Image.new(im.mode, (width, width), BACKGROUND_COLOR)
            result.paste(im, (0, (width - height) // 2))
        elif width < height:
            result = Image.new(im.mode, (height, height), BACKGROUND_COLOR)
            result.paste(im, ((height - width) // 2, 0))
        else:
            result = im
    except OSError as e:
        pass
    result = result.resize((target_size, target_size))
    image_array = img_to_array(result)
    return image_array

print('File: ', image_file_path)
image_array = load_and_preprocess_image(image_file_path, 224)
plt. imshow(image_array.astype('uint8'))
plt.axis('off')
plt.show()

5. Save the 31 models with the BNP cut-off 200 pg/mL under the specified 'model_dir_path'.<br>
Example:<br>
model_dir_path = '/content/drive/MyDrive/BNP_CO_200'<br>

In [4]:
%%time

# feature_branch. DELETE develop or main branch

local_env = True #本番環境ではFalseにする

MODEL_PATH = []
TARGET_SIZE_ARR = []
MODEL_PREPROCESS_INPUT = []

def preprocess_as_itis(x):
    return x

def preprocess_input_mlpmixer(x):
    return x/255

# VGG16 68 ep acc 0.842 pre 0.868 rec 0.805 spec 0.879 f1 0.835 ROC 0.919 PR 0.926
MODEL_NUMBER = 0
if local_env:
    MODEL_PATH.append('C:/Users/egkag/Dropbox/CurrentStudyOnDB/ChestXrayBNP/code1/EKcode1VGG16MishMixupRerasingSAM20220925_2')
else:
    MODEL_PATH.append('/content/drive/MyDrive/BNP_CO_200/model_0.h5')
TARGET_SIZE_ARR.append(224)
MODEL_PREPROCESS_INPUT.append(tf.keras.applications.vgg16.preprocess_input)

# VGG19 170 ep acc 0.838 pre 0.869 rec 0.793 spec 0.882 f1 0.829 ROC 0.913 PR 0.918
MODEL_NUMBER = 1
if local_env:
    MODEL_PATH.append('C:/Users/egkag/Dropbox/CurrentStudyOnDB/ChestXrayBNP/code1/EKVGG19MishMixupRerasingCutmixLabelsmoothSAMRadamCA20221003_3')
else:
    MODEL_PATH.append('/content/drive/MyDrive/BNP_CO_200/model_1.h5')
TARGET_SIZE_ARR.append(224)
MODEL_PREPROCESS_INPUT.append(tf.keras.applications.vgg19.preprocess_input)

# InceptionResNetV2_3 45 ep acc 0.829 pre 0.852 rec 0.793 spec 0.864 f1 0.822 roc 0.909 pr 0.909
MODEL_NUMBER = 2
if local_env:
    MODEL_PATH.append('C:/Users/egkag/Dropbox/CurrentStudyOnDB/ChestXrayBNP/code1/EKcode1InceptionResNetV2_3MishMixupRerasingCutmixLabelsmoothSAMRadamCA20221028')
else:
    MODEL_PATH.append('/content/drive/MyDrive/BNP_CO_200/model_2.h5')
TARGET_SIZE_ARR.append(299)
MODEL_PREPROCESS_INPUT.append(tf.keras.applications.inception_resnet_v2.preprocess_input)

# Xception relu 31 ep acc 0.791 PR *0.916, rec 0.638, spec *0.942, f1 0.752, ROC 0.899, PR 0.904
MODEL_NUMBER = 3
if local_env:
    MODEL_PATH.append('C:/Users/egkag/Dropbox/CurrentStudyOnDB/ChestXrayBNP/code1/EKcode1XceptionReluMixupRerasing20220922_3')
else:
    MODEL_PATH.append('/content/drive/MyDrive/BNP_CO_200/model_3.h5')
TARGET_SIZE_ARR.append(299)
MODEL_PREPROCESS_INPUT.append(tf.keras.applications.xception.preprocess_input)

# Xception 45 ep acc 0.840 PR 0.862, rec 0.807, spec 0.873, f1 0.834, ROC *0.919, PR *0.926
MODEL_NUMBER = 4
if local_env:
    MODEL_PATH.append('C:/Users/egkag/Dropbox/CurrentStudyOnDB/ChestXrayBNP/code1/EKcode1XceptionMishMixupRerasingCutmixLabelsmoothSAMRadamCA20221004_2')
else:
    MODEL_PATH.append('/content/drive/MyDrive/BNP_CO_200/model_4.h5')
TARGET_SIZE_ARR.append(299)
MODEL_PREPROCESS_INPUT.append(tf.keras.applications.xception.preprocess_input)

MODEL_NUMBER = 5
# MobileNetV3Small 86 ep acc 0.809 pre 0.876 rec 0.717 sepc 0.900 f1 0.789 roc 0.914 pr 0.918
if local_env:
    MODEL_PATH.append('C:/Users/egkag/Dropbox/CurrentStudyOnDB/ChestXrayBNP/code1/google_drive_colab/MovileNetV3SmallMishMixupRerasingCutmixLabelsmoothSAMRadamCA20221006_3')
else:
    MODEL_PATH.append('/content/drive/MyDrive/BNP_CO_200/model_5.h5')
TARGET_SIZE_ARR.append(224)
MODEL_PREPROCESS_INPUT.append(preprocess_as_itis)

# MobileNetV3Large_3 76 ep acc 0.837 pre 0.863 rec 0.799 sepc 0.875 f1 0.830 roc 0.912 pr 0.917
MODEL_NUMBER = 6
if local_env:
    MODEL_PATH.append('C:/Users/egkag/Dropbox/CurrentStudyOnDB/ChestXrayBNP/code1/google_drive_colab/EKcode1MobileNetV3LargeMishMixupRerasingCutmixLabelsmoothSAMRadamCA20221023_2')
else:
    MODEL_PATH.append('/content/drive/MyDrive/BNP_CO_200/model_6.h5')
TARGET_SIZE_ARR.append(224)
MODEL_PREPROCESS_INPUT.append(preprocess_as_itis)

MODEL_NUMBER = 7
# ResNet-RS101 78 ep acc 0.823 pre 0.884 rec 0.741 spec 0.904 f1 0.806 roc 0.902 pr 0.909
if local_env:
    MODEL_PATH.append('C:/Users/egkag/Dropbox/CurrentStudyOnDB/ChestXrayBNP/code1/google_drive_colab/EKcode1ResNetRS152MishMixupRerasingCutmixLabelsmoothSAMRadamCA20221022')
else:
    MODEL_PATH.append('/content/drive/MyDrive/BNP_CO_200/model_7.h5')
TARGET_SIZE_ARR.append(224)
MODEL_PREPROCESS_INPUT.append(preprocess_as_itis)

MODEL_NUMBER = 8
# ResNet-RS200 69 ep acc 0.823 pre 0.879 rec 0.745 spc 0.899 f1 0.807 roc 0.903 pr 0.906
if local_env:
    MODEL_PATH.append('C:/Users/egkag/Dropbox/CurrentStudyOnDB/ChestXrayBNP/code1/google_drive_colab/EKcode1ResNetRS200MishMixupRerasingCutmixLabelsmoothSAMRadamCA20221019_4')
else:
    MODEL_PATH.append('/content/drive/MyDrive/BNP_CO_200/model_8.h5')
TARGET_SIZE_ARR.append(224)
MODEL_PREPROCESS_INPUT.append(preprocess_as_itis)

MODEL_NUMBER = 9
# EfficientNetV2B0_2 60 ep acc 0.819 pr 0.841 rec 0.784 spec 0.854 f1 0.811 roc 0.900 pr 0.900
if local_env:
    MODEL_PATH.append('C:/Users/egkag/Dropbox/CurrentStudyOnDB/ChestXrayBNP/code1/google_drive_colab/EKcode1EfficientNetV2B0MishMixupRerasingCutmixLabelsmoothSAMRadamCA20221025')
else:
    MODEL_PATH.append('/content/drive/MyDrive/BNP_CO_200/model_9.h5')
TARGET_SIZE_ARR.append(224)
MODEL_PREPROCESS_INPUT.append(preprocess_as_itis)

MODEL_NUMBER = 10
# EfficientNetV2B1_4 48 ep acc 0.831, pre 0.847, rec 0.806, spec 0.856, f1 0.826, roc 0.904, pr 0.909
if local_env:
    MODEL_PATH.append('C:/Users/egkag/Dropbox/CurrentStudyOnDB/ChestXrayBNP/code1/google_drive_colab/EKcode1EfficientNetV2B1_4MishMixupRerasingCutmixLabelsmoothSAMRadamCA20221105_2')
else:
    MODEL_PATH.append('/content/drive/MyDrive/BNP_CO_200/model_10.h5')
TARGET_SIZE_ARR.append(240)
MODEL_PREPROCESS_INPUT.append(preprocess_as_itis)

MODEL_NUMBER = 11
# EfficientNetV2B2_3 43 ep acc 0.821 pre 0.801 rec 0.852 spec 0.791 f1 0.826 roc 0.907 pre 0.912
if local_env:
    MODEL_PATH.append('C:/Users/egkag/Dropbox/CurrentStudyOnDB/ChestXrayBNP/code1/EKcode1EfficientNetV2B2_3MishMixupRerasingCutmixLabelsmoothSAMRadamCA20221026')
else:
    MODEL_PATH.append('/content/drive/MyDrive/BNP_CO_200/model_11.h5')
TARGET_SIZE_ARR.append(260)
MODEL_PREPROCESS_INPUT.append(preprocess_as_itis)

MODEL_NUMBER = 12
# EfficientNetV2B3_3 39 ep acc 0.825 pre 0.873 rec 0.758 spec 0.891 f1 0.811 roc 0.908 pr 0.908
if local_env:
    MODEL_PATH.append('C:/Users/egkag/Dropbox/CurrentStudyOnDB/ChestXrayBNP/code1/EKcode1EfficientNetV2B3_3MishMixupRerasingCutmixLabelsmoothSAMRadamCA20221102_2')
else:
    MODEL_PATH.append('/content/drive/MyDrive/BNP_CO_200/model_12.h5')
TARGET_SIZE_ARR.append(300)
MODEL_PREPROCESS_INPUT.append(preprocess_as_itis)

MODEL_NUMBER = 13
# EfficientNetV2S_2 40 ep acc 0.844 pre 0.867 rec 0.810 spec 0.878 f1 0.837 roc 0.914 pre 0.919
if local_env:
    MODEL_PATH.append('C:/Users/egkag/Dropbox/CurrentStudyOnDB/ChestXrayBNP/code1/EKcode1EfficientNetV2S_2MishMixupRerasingCutmixLabelsmoothSAMRadamCA20221025_3')
else:
    MODEL_PATH.append('/content/drive/MyDrive/BNP_CO_200/model_13.h5')
TARGET_SIZE_ARR.append(384)
MODEL_PREPROCESS_INPUT.append(preprocess_as_itis)

MODEL_NUMBER = 14
# EfficientNetV2M_2 39 ep acc 0.835 pre 0.853 rec 0.807 spec 0.863 f1 0.830 roc 0.915 pr 0.919
if local_env:
    MODEL_PATH.append('C:/Users/egkag/Dropbox/CurrentStudyOnDB/ChestXrayBNP/code1/google_drive_colab/EKcode1EfficientNetV2M_2MishMixupRerasingCutmixLabelsmoothSAMRadamCA20221104')
else:
    MODEL_PATH.append('/content/drive/MyDrive/BNP_CO_200/model_14.h5')
TARGET_SIZE_ARR.append(480)
MODEL_PREPROCESS_INPUT.append(preprocess_as_itis)

MODEL_NUMBER = 15
# EfficientNetV2L 42 ep acc 0.832 pre 0.839 rec 0.819 spec 0.845 f1 b0.829 roc 0.909 pr 0.906
if local_env:
    MODEL_PATH.append('C:/Users/egkag/Dropbox/CurrentStudyOnDB/ChestXrayBNP/code1/EKcode1EfficientNetV2LMishMixupRerasingCutmixLabelsmoothSAMRadamCA20221012_2')
else:
    MODEL_PATH.append('/content/drive/MyDrive/BNP_CO_200/model_15.h5')
TARGET_SIZE_ARR.append(480)
MODEL_PREPROCESS_INPUT.append(preprocess_as_itis)

MODEL_NUMBER = 16
# ConvNeXtTiny 52 ep acc 0.831 pre 0.865 rec 0.781 spec 0.879 f1 0.821 roc 0.922 pr 0.924
if local_env:
    MODEL_PATH.append('C:/Users/egkag/Dropbox/CurrentStudyOnDB/ChestXrayBNP/code1/EKcode1ConvNeXtTinyMishMixupRerasingCutmixLabelsmoothSAMRadamCA20221010_3')
else:
    MODEL_PATH.append('/content/drive/MyDrive/BNP_CO_200/model_16.h5')
TARGET_SIZE_ARR.append(224)
MODEL_PREPROCESS_INPUT.append(preprocess_as_itis)

MODEL_NUMBER = 17
# ConvNeXtSmall 46 ep acc 0.827 pre 0.866 rec 0.772 sepc 0.882 f1 0.816 roc 0.918 pr 0.922
if local_env:
    MODEL_PATH.append('C:/Users/egkag/Dropbox/CurrentStudyOnDB/ChestXrayBNP/code1/EKcode1ConvNeXtSmallMishMixupRerasingCutmixLabelsmoothSAMRadamCA20221006_2')
else:
    MODEL_PATH.append('/content/drive/MyDrive/BNP_CO_200/model_17.h5')
TARGET_SIZE_ARR.append(224)
MODEL_PREPROCESS_INPUT.append(preprocess_as_itis)

MODEL_NUMBER = 18
# ConvNeXtBase 41 ep acc 0.830 pre 0.834 rec 0.820 spec 0.839 f1 0.827 roc 0.910 pre 0.913
if local_env:
    MODEL_PATH.append('C:/Users/egkag/Dropbox/CurrentStudyOnDB/ChestXrayBNP/code1/EKcode1ConvNeXtBaseMishMixupRerasingCutmixLabelsmoothSAMRadamCA20221004_3')
else:
    MODEL_PATH.append('/content/drive/MyDrive/BNP_CO_200/model_18.h5')
TARGET_SIZE_ARR.append(224)
MODEL_PREPROCESS_INPUT.append(preprocess_as_itis)

MODEL_NUMBER = 19
# ConvNeXtLarge_3 36 ep acc 0.832 pre 0.835 rec 0.825 spec 0.839 f1 0.830 roc 0.906 pr 0.908
if local_env:
    MODEL_PATH.append('C:/Users/egkag/Dropbox/CurrentStudyOnDB/ChestXrayBNP/code1/EKcode1ConvNeXtLarge_3MishMixupRerasingCutmixLabelsmoothSAMRadamCA20221024_2')
else:
    MODEL_PATH.append('/content/drive/MyDrive/BNP_CO_200/model_19.h5')
TARGET_SIZE_ARR.append(224)
MODEL_PREPROCESS_INPUT.append(preprocess_as_itis)

MODEL_NUMBER = 20
# ConvNeXtXLarge 37 ep acc 0.820 pre 0.889 rec 0.727 spec 0.911 f1 0.800 roc 0.913 pre 0.919
if local_env:
    MODEL_PATH.append('C:/Users/egkag/Dropbox/CurrentStudyOnDB/ChestXrayBNP/code1/EKcode1ConvNeXtXLargeMishMixupRerasingCutmixLabelsmoothSAMRadamCA20221007_5')
else:
    MODEL_PATH.append('/content/drive/MyDrive/BNP_CO_200/model_20.h5')
TARGET_SIZE_ARR.append(224)
MODEL_PREPROCESS_INPUT.append(preprocess_as_itis)

MODEL_NUMBER = 21
# Vit-b16 384 116 ep acc 0.828 pre 0.903 rec 0.733 spec 0.922 f1 0.809 oroc 0.919 pre 0.924
if local_env:
    MODEL_PATH.append('C:/Users/egkag/Dropbox/CurrentStudyOnDB/ChestXrayBNP/code1/google_drive_colab/EKcode1VitB16MishMixupRerasingCutmixLabelsmoothSAMRadamCA20221012_100_150best')
else:
    MODEL_PATH.append('/content/drive/MyDrive/BNP_CO_200/model_21.h5')
TARGET_SIZE_ARR.append(384)
MODEL_PREPROCESS_INPUT.append(preprocess_inputs)

MODEL_NUMBER = 22
# Vit-b16 320 43 ep acc 0.811 pre 0.863 rec 0.7365 pspec 0.885 f1 0.794 roc 0.905 pr 0.909
if local_env:
    MODEL_PATH.append('C:/Users/egkag/Dropbox/CurrentStudyOnDB/ChestXrayBNP/code1/EKcode1VitB16_3MishMixupRerasingCutmixLabelsmoothSAMRadamCA20221026_2')
else:
    MODEL_PATH.append('/content/drive/MyDrive/BNP_CO_200/model_22.h5')
TARGET_SIZE_ARR.append(320)
MODEL_PREPROCESS_INPUT.append(preprocess_inputs)

MODEL_NUMBER = 23
# Vit-b16 224 150 ep acc 0.827 pre 0.816 rec 0.843 sepc 0.812 f1 0.829 roc 0.906 pre 0.912
if local_env:
    MODEL_PATH.append('C:/Users/egkag/Dropbox/CurrentStudyOnDB/ChestXrayBNP/code1/google_drive_colab/EKcode1VitB16_2MishMixupRerasingCutmixLabelsmoothSAMRadamCA20221024_3_150ep')
else:
    MODEL_PATH.append('/content/drive/MyDrive/BNP_CO_200/model_23.h5')
TARGET_SIZE_ARR.append(224)
MODEL_PREPROCESS_INPUT.append(preprocess_inputs)

MODEL_NUMBER = 24
# MLPMixerB32 66 ep acc 0.817 pre 0.856 rec 0.759 spec 0.874 f1 0.804 roc 0.901 pre 0.898
if local_env:
    MODEL_PATH.append('C:/Users/egkag/Dropbox/CurrentStudyOnDB/ChestXrayBNP/code1/EKcode1MLPmixerb32_2MishMixupRerasingCutmixLabelsmoothSAMRadamCA20221021_2')
else:
    MODEL_PATH.append('/content/drive/MyDrive/BNP_CO_200/model_24.h5')
TARGET_SIZE_ARR.append(224)
MODEL_PREPROCESS_INPUT.append(preprocess_input_mlpmixer)

MODEL_NUMBER = 25
# MLPMixerB32_3 200 ep acc 0.768 pre 0.716 rec 0.884 spec 0.653 f1 0.791 roc 0.864 pr 0.862
if local_env:
    MODEL_PATH.append('C:/Users/egkag/Dropbox/CurrentStudyOnDB/ChestXrayBNP/code1/google_drive_colab/EKcode1MLPMixerB32_3MishMixupRerasingCutmixLabelsmoothSAMRadamCA20221026_3_200ep')
else:
    MODEL_PATH.append('/content/drive/MyDrive/BNP_CO_200/model_25.h5')
TARGET_SIZE_ARR.append(224)
MODEL_PREPROCESS_INPUT.append(preprocess_input_mlpmixer)

MODEL_NUMBER = 26
# VGG16_2 384*384*3 210 ep acc 0.823 pre 0.877 rec 0.749 apec 0.897 f1 0.808 roc 0.907 pr 0.914
if local_env:
    MODEL_PATH.append('C:/Users/egkag/Dropbox/CurrentStudyOnDB/ChestXrayBNP/code1/EKcode1VGG16_2MishMixupRerasingCutmixLabelsmoothSAMRadamCA20221031_2_200_250best')
else:
    MODEL_PATH.append('/content/drive/MyDrive/BNP_CO_200/model_26.h5')
TARGET_SIZE_ARR.append(384)
MODEL_PREPROCESS_INPUT.append(tf.keras.applications.vgg16.preprocess_input)

MODEL_NUMBER = 27
# MLPMixerB32_3 150 ep acc 0.605 pre 0.558 rec 0.985 spec 0.231 roc 0.834 pr 0.817
if local_env:
    MODEL_PATH.append('C:/Users/egkag/Dropbox/CurrentStudyOnDB/ChestXrayBNP/code1/google_drive_colab/EKcode1MLPMixerB32_3MishMixupRerasingCutmixLabelsmoothSAMRadamCA20221026_3_150ep')
else:
    MODEL_PATH.append('/content/drive/MyDrive/BNP_CO_200/model_27.h5')
TARGET_SIZE_ARR.append(224)
MODEL_PREPROCESS_INPUT.append(preprocess_input_mlpmixer)

MODEL_NUMBER = 28
# Vit-b16 224 100 ep acc 0.822 pre 0.816 rec 0.828 spec 0.816 f1 0.822 roc 0.906 prr 0.909
if local_env:
    MODEL_PATH.append('C:/Users/egkag/Dropbox/CurrentStudyOnDB/ChestXrayBNP/code1/google_drive_colab/EKcode1VitB16_2MishMixupRerasingCutmixLabelsmoothSAMRadamCA20221024_3_100ep')
else:
    MODEL_PATH.append('/content/drive/MyDrive/BNP_CO_200/model_28.h5')
TARGET_SIZE_ARR.append(224)
MODEL_PREPROCESS_INPUT.append(preprocess_inputs)

MODEL_NUMBER = 29
# VGG19_2 384 53 ep acc 0.841 pre 0.871 rec 0.799 spec 0.883 f1 0.833 roc 0.915 pr 0.923
if local_env:
    MODEL_PATH.append('C:/Users/egkag/Dropbox/CurrentStudyOnDB/ChestXrayBNP/code1/google_drive_colab/EKcode1VGG19_2MishMixupRerasingCutmixLabelsmoothSAMRadamCA20221103')
else:
    MODEL_PATH.append('/content/drive/MyDrive/BNP_CO_200/model_29.h5')
TARGET_SIZE_ARR.append(384)
MODEL_PREPROCESS_INPUT.append(tf.keras.applications.vgg19.preprocess_input)

MODEL_NUMBER = 30
# VGG19_2 384 100 ep acc 0.834 pre 0.844 rec 0.816 spec 0.852 f1 0.830 roc 0.910 pr 0.917
if local_env:
    MODEL_PATH.append('C:/Users/egkag/Dropbox/CurrentStudyOnDB/ChestXrayBNP/code1/google_drive_colab/EKcode1VGG19_2MishMixupRerasingCutmixLabelsmoothSAMRadamCA20221103_100ep')
else:
    MODEL_PATH.append('/content/drive/MyDrive/BNP_CO_200/model_30.h5')
TARGET_SIZE_ARR.append(384)
MODEL_PREPROCESS_INPUT.append(tf.keras.applications.vgg19.preprocess_input)

MODEL_N = len(MODEL_PATH)
MODEL_MAJORITY_N = (MODEL_N + 1) // 2
print('Model N = ', MODEL_N)

for i in range(MODEL_N):
    if os.path.exists(MODEL_PATH[i]) == False:
        print(f'There is no directory {MODEL_PATH[i]}.')

Model N =  31
CPU times: total: 0 ns
Wall time: 1e+03 µs


In [ ]:
%%time

MODEL_PATH = []
TARGET_SIZE_ARR = []
MODEL_PREPROCESS_INPUT = []

def preprocess_as_itis(x):
    return x

def preprocess_input_mlpmixer(x):
    return x/255

# VGG16 68 ep acc 0.842 pre 0.868 rec 0.805 spec 0.879 f1 0.835 ROC 0.919 PR 0.926
MODEL_NUMBER = 0
MODEL_PATH.append('/content/drive/MyDrive/BNP_CO_200/model_0.h5')
TARGET_SIZE_ARR.append(224)
MODEL_PREPROCESS_INPUT.append(tf.keras.applications.vgg16.preprocess_input)

# VGG19 170 ep acc 0.838 pre 0.869 rec 0.793 spec 0.882 f1 0.829 ROC 0.913 PR 0.918
MODEL_NUMBER = 1
MODEL_PATH.append('/content/drive/MyDrive/BNP_CO_200/model_1.h5')
TARGET_SIZE_ARR.append(224)
MODEL_PREPROCESS_INPUT.append(tf.keras.applications.vgg19.preprocess_input)

# InceptionResNetV2_3 45 ep acc 0.829 pre 0.852 rec 0.793 spec 0.864 f1 0.822 roc 0.909 pr 0.909
MODEL_NUMBER = 2
MODEL_PATH.append('/content/drive/MyDrive/BNP_CO_200/model_2.h5')
TARGET_SIZE_ARR.append(299)
MODEL_PREPROCESS_INPUT.append(tf.keras.applications.inception_resnet_v2.preprocess_input)

# Xception relu 31 ep acc 0.791 PR *0.916, rec 0.638, spec *0.942, f1 0.752, ROC 0.899, PR 0.904
MODEL_NUMBER = 3
MODEL_PATH.append('/content/drive/MyDrive/BNP_CO_200/model_3.h5')
TARGET_SIZE_ARR.append(299)
MODEL_PREPROCESS_INPUT.append(tf.keras.applications.xception.preprocess_input)

# Xception 45 ep acc 0.840 PR 0.862, rec 0.807, spec 0.873, f1 0.834, ROC *0.919, PR *0.926
MODEL_NUMBER = 4
MODEL_PATH.append('/content/drive/MyDrive/BNP_CO_200/model_4.h5')
TARGET_SIZE_ARR.append(299)
MODEL_PREPROCESS_INPUT.append(tf.keras.applications.xception.preprocess_input)

MODEL_NUMBER = 5
# MobileNetV3Small 86 ep acc 0.809 pre 0.876 rec 0.717 sepc 0.900 f1 0.789 roc 0.914 pr 0.918
MODEL_PATH.append('/content/drive/MyDrive/BNP_CO_200/model_5.h5')
TARGET_SIZE_ARR.append(224)
MODEL_PREPROCESS_INPUT.append(preprocess_as_itis)

# MobileNetV3Large_3 76 ep acc 0.837 pre 0.863 rec 0.799 sepc 0.875 f1 0.830 roc 0.912 pr 0.917
MODEL_NUMBER = 6
MODEL_PATH.append('/content/drive/MyDrive/BNP_CO_200/model_6.h5')
TARGET_SIZE_ARR.append(224)
MODEL_PREPROCESS_INPUT.append(preprocess_as_itis)

MODEL_NUMBER = 7
# ResNet-RS101 78 ep acc 0.823 pre 0.884 rec 0.741 spec 0.904 f1 0.806 roc 0.902 pr 0.909
MODEL_PATH.append('/content/drive/MyDrive/BNP_CO_200/model_7.h5')
TARGET_SIZE_ARR.append(224)
MODEL_PREPROCESS_INPUT.append(preprocess_as_itis)

MODEL_NUMBER = 8
# ResNet-RS200 69 ep acc 0.823 pre 0.879 rec 0.745 spc 0.899 f1 0.807 roc 0.903 pr 0.906
MODEL_PATH.append('/content/drive/MyDrive/BNP_CO_200/model_8.h5')
TARGET_SIZE_ARR.append(224)
MODEL_PREPROCESS_INPUT.append(preprocess_as_itis)

MODEL_NUMBER = 9
# EfficientNetV2B0_2 60 ep acc 0.819 pr 0.841 rec 0.784 spec 0.854 f1 0.811 roc 0.900 pr 0.900
MODEL_PATH.append('/content/drive/MyDrive/BNP_CO_200/model_9.h5')
TARGET_SIZE_ARR.append(224)
MODEL_PREPROCESS_INPUT.append(preprocess_as_itis)

MODEL_NUMBER = 10
# EfficientNetV2B1_4 48 ep acc 0.831, pre 0.847, rec 0.806, spec 0.856, f1 0.826, roc 0.904, pr 0.909
MODEL_PATH.append('/content/drive/MyDrive/BNP_CO_200/model_10.h5')
TARGET_SIZE_ARR.append(240)
MODEL_PREPROCESS_INPUT.append(preprocess_as_itis)

MODEL_NUMBER = 11
# EfficientNetV2B2_3 43 ep acc 0.821 pre 0.801 rec 0.852 spec 0.791 f1 0.826 roc 0.907 pre 0.912
MODEL_PATH.append('/content/drive/MyDrive/BNP_CO_200/model_11.h5')
TARGET_SIZE_ARR.append(260)
MODEL_PREPROCESS_INPUT.append(preprocess_as_itis)

MODEL_NUMBER = 12
# EfficientNetV2B3_3 39 ep acc 0.825 pre 0.873 rec 0.758 spec 0.891 f1 0.811 roc 0.908 pr 0.908
MODEL_PATH.append('/content/drive/MyDrive/BNP_CO_200/model_12.h5')
TARGET_SIZE_ARR.append(300)
MODEL_PREPROCESS_INPUT.append(preprocess_as_itis)

MODEL_NUMBER = 13
# EfficientNetV2S_2 40 ep acc 0.844 pre 0.867 rec 0.810 spec 0.878 f1 0.837 roc 0.914 pre 0.919
MODEL_PATH.append('/content/drive/MyDrive/BNP_CO_200/model_13.h5')
TARGET_SIZE_ARR.append(384)
MODEL_PREPROCESS_INPUT.append(preprocess_as_itis)

MODEL_NUMBER = 14
# EfficientNetV2M_2 39 ep acc 0.835 pre 0.853 rec 0.807 spec 0.863 f1 0.830 roc 0.915 pr 0.919
MODEL_PATH.append('/content/drive/MyDrive/BNP_CO_200/model_14.h5')
TARGET_SIZE_ARR.append(480)
MODEL_PREPROCESS_INPUT.append(preprocess_as_itis)

MODEL_NUMBER = 15
# EfficientNetV2L 42 ep acc 0.832 pre 0.839 rec 0.819 spec 0.845 f1 b0.829 roc 0.909 pr 0.906
MODEL_PATH.append('/content/drive/MyDrive/BNP_CO_200/model_15.h5')
TARGET_SIZE_ARR.append(480)
MODEL_PREPROCESS_INPUT.append(preprocess_as_itis)

MODEL_NUMBER = 16
# ConvNeXtTiny 52 ep acc 0.831 pre 0.865 rec 0.781 spec 0.879 f1 0.821 roc 0.922 pr 0.924
MODEL_PATH.append('/content/drive/MyDrive/BNP_CO_200/model_16.h5')
TARGET_SIZE_ARR.append(224)
MODEL_PREPROCESS_INPUT.append(preprocess_as_itis)

MODEL_NUMBER = 17
# ConvNeXtSmall 46 ep acc 0.827 pre 0.866 rec 0.772 sepc 0.882 f1 0.816 roc 0.918 pr 0.922
MODEL_PATH.append('/content/drive/MyDrive/BNP_CO_200/model_17.h5')
TARGET_SIZE_ARR.append(224)
MODEL_PREPROCESS_INPUT.append(preprocess_as_itis)

MODEL_NUMBER = 18
# ConvNeXtBase 41 ep acc 0.830 pre 0.834 rec 0.820 spec 0.839 f1 0.827 roc 0.910 pre 0.913
MODEL_PATH.append('/content/drive/MyDrive/BNP_CO_200/model_18.h5')
TARGET_SIZE_ARR.append(224)
MODEL_PREPROCESS_INPUT.append(preprocess_as_itis)

MODEL_NUMBER = 19
# ConvNeXtLarge_3 36 ep acc 0.832 pre 0.835 rec 0.825 spec 0.839 f1 0.830 roc 0.906 pr 0.908
MODEL_PATH.append('/content/drive/MyDrive/BNP_CO_200/model_19.h5')
TARGET_SIZE_ARR.append(224)
MODEL_PREPROCESS_INPUT.append(preprocess_as_itis)

MODEL_NUMBER = 20
# ConvNeXtXLarge 37 ep acc 0.820 pre 0.889 rec 0.727 spec 0.911 f1 0.800 roc 0.913 pre 0.919
MODEL_PATH.append('/content/drive/MyDrive/BNP_CO_200/model_20.h5')
TARGET_SIZE_ARR.append(224)
MODEL_PREPROCESS_INPUT.append(preprocess_as_itis)

MODEL_NUMBER = 21
# Vit-b16 384 116 ep acc 0.828 pre 0.903 rec 0.733 spec 0.922 f1 0.809 oroc 0.919 pre 0.924
MODEL_PATH.append('/content/drive/MyDrive/BNP_CO_200/model_21.h5')
TARGET_SIZE_ARR.append(384)
MODEL_PREPROCESS_INPUT.append(preprocess_inputs)

MODEL_NUMBER = 22
# Vit-b16 320 43 ep acc 0.811 pre 0.863 rec 0.7365 pspec 0.885 f1 0.794 roc 0.905 pr 0.909
MODEL_PATH.append('/content/drive/MyDrive/BNP_CO_200/model_22.h5')
TARGET_SIZE_ARR.append(320)
MODEL_PREPROCESS_INPUT.append(preprocess_inputs)

MODEL_NUMBER = 23
# Vit-b16 224 150 ep acc 0.827 pre 0.816 rec 0.843 sepc 0.812 f1 0.829 roc 0.906 pre 0.912
MODEL_PATH.append('/content/drive/MyDrive/BNP_CO_200/model_23.h5')
TARGET_SIZE_ARR.append(224)
MODEL_PREPROCESS_INPUT.append(preprocess_inputs)

MODEL_NUMBER = 24
# MLPMixerB32 66 ep acc 0.817 pre 0.856 rec 0.759 spec 0.874 f1 0.804 roc 0.901 pre 0.898
MODEL_PATH.append('/content/drive/MyDrive/BNP_CO_200/model_24.h5')
TARGET_SIZE_ARR.append(224)
MODEL_PREPROCESS_INPUT.append(preprocess_input_mlpmixer)

MODEL_NUMBER = 25
# MLPMixerB32_3 200 ep acc 0.768 pre 0.716 rec 0.884 spec 0.653 f1 0.791 roc 0.864 pr 0.862
MODEL_PATH.append('/content/drive/MyDrive/BNP_CO_200/model_25.h5')
TARGET_SIZE_ARR.append(224)
MODEL_PREPROCESS_INPUT.append(preprocess_input_mlpmixer)

MODEL_NUMBER = 26
# VGG16_2 384*384*3 210 ep acc 0.823 pre 0.877 rec 0.749 apec 0.897 f1 0.808 roc 0.907 pr 0.914
MODEL_PATH.append('/content/drive/MyDrive/BNP_CO_200/model_26.h5')
TARGET_SIZE_ARR.append(384)
MODEL_PREPROCESS_INPUT.append(tf.keras.applications.vgg16.preprocess_input)

MODEL_NUMBER = 27
# MLPMixerB32_3 150 ep acc 0.605 pre 0.558 rec 0.985 spec 0.231 roc 0.834 pr 0.817
MODEL_PATH.append('/content/drive/MyDrive/BNP_CO_200/model_27.h5')
TARGET_SIZE_ARR.append(224)
MODEL_PREPROCESS_INPUT.append(preprocess_input_mlpmixer)

MODEL_NUMBER = 28
# Vit-b16 224 100 ep acc 0.822 pre 0.816 rec 0.828 spec 0.816 f1 0.822 roc 0.906 prr 0.909
MODEL_PATH.append('/content/drive/MyDrive/BNP_CO_200/model_28.h5')
TARGET_SIZE_ARR.append(224)
MODEL_PREPROCESS_INPUT.append(preprocess_inputs)

MODEL_NUMBER = 29
# VGG19_2 384 53 ep acc 0.841 pre 0.871 rec 0.799 spec 0.883 f1 0.833 roc 0.915 pr 0.923
MODEL_PATH.append('/content/drive/MyDrive/BNP_CO_200/model_29.h5')
TARGET_SIZE_ARR.append(384)
MODEL_PREPROCESS_INPUT.append(tf.keras.applications.vgg19.preprocess_input)

MODEL_NUMBER = 30
# VGG19_2 384 100 ep acc 0.834 pre 0.844 rec 0.816 spec 0.852 f1 0.830 roc 0.910 pr 0.917
MODEL_PATH.append('/content/drive/MyDrive/BNP_CO_200/model_30.h5')
TARGET_SIZE_ARR.append(384)
MODEL_PREPROCESS_INPUT.append(tf.keras.applications.vgg19.preprocess_input)

MODEL_N = len(MODEL_PATH)
MODEL_MAJORITY_N = (MODEL_N + 1) // 2
print('Model N = ', MODEL_N)

for i in range(MODEL_N):
    if os.path.exists(MODEL_PATH[i]) == False:
        print(f'There is no directory {MODEL_PATH[i]}.')

6. Ensemble prediction (31 models).<br>
The ensemble results will be output sequentially up to the model that has been executed.<br>
Execution may be interrupted if there is insufficient memory or other issues.<br>

In [5]:
%%time

probs, preds = np.zeros(MODEL_N), np.zeros(MODEL_N)
preds_pos_num = 0
custom_objects = {"Addons>mish": tfa.activations.mish}
for i in range(MODEL_N):
    print('Model No.', i)

    image_array = load_and_preprocess_image(image_file_path, TARGET_SIZE_ARR[i])
    print('Shape of image: ', image_array.shape)

    model = tf.keras.models.load_model(MODEL_PATH[i], custom_objects=custom_objects)
    img_array = np.expand_dims(image_array, axis=0)
    preprocess_input = MODEL_PREPROCESS_INPUT[i]
    tmp_probs = model.predict(preprocess_input(img_array))
    probs[i] = tmp_probs[0][1]
    if probs[i] < 0.5:
        preds[i] = 0
    else:
        preds[i] = 1
    print('Single model: ', MODEL_PATH[i])
    print('The provability of BNP >= 200 pg/mL (single model):', probs[i])
    print('Soft ensemble provability: ', np.sum(probs) / float(i + 1))
    print('Hard ensemble: ', int(np.sum(preds)), '/', i + 1, '=', np.sum(preds) / float(i+1), '\n')

print('Calculated 31 models.')

Model No. 0
Shape of image:  (224, 224, 3)
1/1 [==============================] - 2s 2s/step
Single model:  C:/Users/egkag/Dropbox/CurrentStudyOnDB/ChestXrayBNP/code1/EKcode1VGG16MishMixupRerasingSAM20220925_2
The provability of BNP >= 200 pg/mL (single model): 0.2131868451833725
Soft ensemble provability:  0.2131868451833725
Hard ensemble:  0 / 1 = 0.0 

Model No. 1
Shape of image:  (224, 224, 3)
1/1 [==============================] - 0s 82ms/step
Single model:  C:/Users/egkag/Dropbox/CurrentStudyOnDB/ChestXrayBNP/code1/EKVGG19MishMixupRerasingCutmixLabelsmoothSAMRadamCA20221003_3
The provability of BNP >= 200 pg/mL (single model): 0.43293648958206177
Soft ensemble provability:  0.32306166738271713
Hard ensemble:  0 / 2 = 0.0 

Model No. 2
Shape of image:  (299, 299, 3)
1/1 [==============================] - 2s 2s/step
Single model:  C:/Users/egkag/Dropbox/CurrentStudyOnDB/ChestXrayBNP/code1/EKcode1InceptionResNetV2_3MishMixupRerasingCutmixLabelsmoothSAMRadamCA20221028
The provability

: 